In [1]:
from bs4 import BeautifulSoup as bs
import urllib
import re
import string
import pandas as pd
from operator import itemgetter

In [2]:
bi = pd.read_csv('../Data/List_of_British_Invasion_artists.csv')

In [3]:
bi_list = list(bi.artist.unique())

In [4]:
charts = pd.read_csv('../Data/1964_1969_Billboard_Charts.csv', encoding='utf-8')

In [5]:
chart_list = list(charts.artist.unique())

In [6]:
artists_to_fetch = []
for item in chart_list:
    if item in bi_list:
        pass
    else:
        artists_to_fetch.append(item)

In [7]:
len(chart_list)

1305

In [8]:
len(artists_to_fetch)
charts = charts[charts.artist.isin(artists_to_fetch)]

In [9]:
import pandasql as ps

In [10]:
q = '''
    select artist, count(song) as total_top_100 from
    (select distinct artist, song from charts)
    group by artist 
    order by count(song) asc
    '''

In [11]:
Artists_total_100 = ps.sqldf(q, globals())

In [12]:
artists_to_fetch = Artists_total_100["artist"][Artists_total_100["total_top_100"] > 10].values

In [13]:
len(artists_to_fetch)

87

In [14]:
def url(artist):
    base = "http://www.lyricsfreak.com/"
    name = artist.lower().strip().translate(None, string.punctuation).split(" ")
    if '' in name:
        name.remove('')
    init = name[0][0]
    search_name = "+".join(name)
    URL = base + init + "/" + search_name + "/"
    return URL

def fetch(artist):
    base = "http://www.lyricsfreak.com"
    song_list = []
    URL = url(artist)
    page = urllib.urlopen(URL)
    soup = bs(page, "lxml")
    songs = soup.findAll("table", attrs={"name": "song"})[0].findAll("tr")
    for s in songs:
        for n in s.findAll("td", attrs={"class": "colfirst"}):
            name = re.sub(" Lyrics","",n.find('a').attrs["title"])
            lyric_link = base + n.find('a').attrs["href"]
            song_list.append({name:lyric_link})
    return song_list

def fetch_all_artists(artist_list):
    lists = {}
    for i, artist in enumerate(artist_list):
        # print artist
        # print "\r{}%".format(100.0*(i+1)/len(artist_list)),
        song_list = []
        if artist[:3].lower() == 'the':
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(artist[4:])
            except IndexError:
                pass
        elif 'and' in artist.split(" "):
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(re.sub(' and ', ' & ', artist))
            except IndexError:
                pass
        else:
            try:
                song_list = fetch(artist)
            except:
                print "[Warning!] {}: Not Exist".format(artist)
        lists[artist] = song_list
    return lists

def ly_correct_url(URL):
    # input: lyric_page_url
    # output: lyric_correct_url
    page = urllib.urlopen(URL)
    soup = bs(page, 'lxml')
    base = 'http://www.lyricsfreak.com'
    try:
        correct_url = base + soup.find_all('a', attrs={'class': 'lb-correct'})[0].attrs['href']
    except IndexError:
        correct_url = None
    return correct_url

def fetch_lyrics(URL):
    # input: lyric_page_url
    URL2 = ly_correct_url(URL)
    if URL2:
        page = urllib.urlopen(URL2)
        soup = bs(page, 'lxml')
        lyric_text = soup.findAll('textarea', attrs={'name': 'lyrics'})[0].text
        return lyric_text
    else:
        return None

def fetch_one_artist_all_lyrics(artist):
    li =  [artist] #;print li
    d = fetch_all_artists(li) #;print d
    song_list = d[artist]
    lyrics_list = {} # {name: lyrics}
    for i, song in enumerate(song_list):
        # print '\r{}%'.format(100.0*(i+1)/len(song_list)),
        name = song.keys()[0] #;print name
        url = song.values()[0] #;print url
        lyrics = fetch_lyrics(url)
        lyrics_list[name] = lyrics
    return lyrics_list

def fetch_all_artists_all_lyrics(artist_list):
    artists_songs_lyrics = {} # {artist: {song: lyrics}}
    for i, artist in enumerate(artist_list):
        songs_lyrics = fetch_one_artist_all_lyrics(artist)
        artists_songs_lyrics[artist] = songs_lyrics
        print '\r{}%'.format(100.0*(i+1)/len(artist_list)),
    return artists_songs_lyrics

In [33]:
a = artists_to_fetch
a = map(lambda x: str(x), a)

In [45]:
res = fetch_all_artists_all_lyrics(a)

28.7356321839% [Warning!] Trini Lopez: Not Exist
34.4827586207%     [Warning!] Paul Revere & The Raiders Featuring Mark Lindsay: Not Exist
85.0574712644% [Warning!] James Brown And The Famous Flames: Not Exist
95.4022988506% [Warning!] Elvis Presley With The Jordanaires: Not Exist
100.0%5747126%   


In [60]:
import cPickle as pickle
with open("./Billboard_Songs.pkl", 'wb') as fi:
    pickle.dump(res, fi)

In [61]:
with open("./Billboard_Songs.pkl", 'r') as fi:
    ly_bb = pickle.load(fi)

def dict2df(ly):
    df = pd.DataFrame(columns=['Artist', 'Song', 'Lyrics'])
    for a in ly:
        artist = a
        song_list = ly[a]
        if isinstance(song_list, dict):
            for s in song_list:
                song = s
                lyrics = song_list[s]
                record = pd.DataFrame.from_dict({'Artist': artist,
                                                 'Song': song,
                                                 'Lyrics': lyrics}
                                                ,orient='index').T
                df = df.append(record)  
    df = df.loc[:,['Artist', 'Song', 'Lyrics']]
    df = df.reset_index(drop=True)
    return df

df_bb = dict2df(ly_bb)

In [64]:
df_bb.to_csv('../Data/Contemporary_Artists_Song_Lyrics.csv', encoding='utf-8', quoting=2)